# Summarization

In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("Someman/hindi-summarization")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/55226 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13807 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/17265 [00:00<?, ? examples/s]

In [3]:
print(len(dataset['train']))
dataset_train=dataset['train']
dataset_valid=dataset['valid']
dataset

55226


DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 55226
    })
    test: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 13807
    })
    valid: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 17265
    })
})

In [4]:
num_samples = len(dataset)
samples = int(num_samples * 0.7)
# dataset = dataset_train.take(samples)

from datasets import Dataset
sliced_dataset = dataset['train'][0:35000]
sliced_dataset_valid=dataset['valid'][0:13000]
# dataset_train=dataset_train[:samples]
# dataset_train = Dataset.from_dict(sliced_dataset)
# dataset_valid=Dataset.from_dict(sliced_dataset_valid)

dataset_train = dataset_train
dataset_valid=dataset_valid

In [5]:
# import random
# def show_random(sample=3,seed=42):
#     random_index=random.randint(0, len(dataset_train)-5)

#     for i in range(sample):
#         print("\nHeadline\n",dataset['train']['headline'][random_index+i],"\n")
#         print("\nsummary\n",dataset['train']['summary'][random_index+i])
#         # print("article\n",dataset['train']['article'][random_index+i])

# show_random()

In [6]:
!pip install sentencepiece


In [7]:
# from transformers import AutoTokenizer

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
# tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["input_ids"],
#     shuffle=True,
#     collate_fn=data_collator,
#     batch_size=8,
# )

# tf_validation_dataset = tokenized_datasets["valid"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["input_ids"],
#     shuffle=False,
#     collate_fn=data_collator,
#     batch_size=8,
# )

#Facebook bart

In [9]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
from transformers import TFAutoModelForSeq2SeqLM



model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [10]:
max_input_length = 512
max_target_length = 30
# headline', 'summary', 'article'

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["headline"],examples['article'],
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True,padding=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_datasets = dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_valid = dataset_valid.map(preprocess_function, batched=True)


Map:   0%|          | 0/55226 [00:00<?, ? examples/s]

Map:   0%|          | 0/17265 [00:00<?, ? examples/s]

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [14]:
# from transformers import TFAutoModelForSeq2SeqLM

# model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

In [15]:
# tokenized_datasets = tokenized_datasets.remove_columns(
#     dataset["train"].column_names
# )

In [16]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets_valid,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 2
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [19]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=f"{model_name}-hindi-summary", tokenizer=tokenizer
)

model.fit(

    tf_train_dataset,validation_data=tf_eval_dataset,callbacks=[callback], epochs=2
)

Cloning https://huggingface.co/Henil1/mt5-small-hindi-summary into local empty directory.


Download file tf_model.h5:   0%|          | 17.0k/2.07G [00:00<?, ?B/s]

Download file spiece.model:   0%|          | 7.40k/4.11M [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 8.18k/15.6M [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/4.11M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/15.6M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/2.07G [00:00<?, ?B/s]

Epoch 1/2
6903/6903 [==============================] - 4024s 580ms/step - loss: 3.3876 - val_loss: 1.8697
Epoch 2/2
6903/6903 [==============================] - 3799s 550ms/step - loss: 2.4906 - val_loss: 1.8697


In [20]:
hed="पर्यावरण संरक्षण के महत्वपूर्ण कदम"


art="आजकल की तेजी से बदलती जीवनशैली में पर्यावरण संरक्षण का महत्वपूर्ण योगदान है। अव्यवस्थित और अति उपयोग से प्राकृतिक संसाधनों की दिक्कतें बढ़ रही हैं। जलवायु परिवर्तन, वनस्पति और प्राणियों के लिए खतरे, और विभिन्न प्रदूषण के परिणामस्वरूप यह समस्याएं और भी गंभीर हो रही हैं। पर्यावरण संरक्षण महत्वपूर्ण होने का कारण है कि हमारी सामाजिक और आर्थिक विकास में इसका सीधा संबंध है। स्वच्छता, जल संसाधनों की सुरक्षा, वन्यजीवों के संरक्षण, और वनस्पति के संरक्षण से हमारे प्राकृतिक संसाधन और जीवन की सुरक्षा में सुधार हो सकता है। यह आवश्यक है कि हम सभी समृद्धि के लिए प्राकृतिक संसाधनों का ठीक से उपयोग करें और उन्हें अच्छे से प्रबंधित रखें। यह हमारे आने वाले पीढ़ियों के लिए भी महत्वपूर्ण है ताकि उन्हें भी स्वस्थ और सुरक्षित पर्यावरण मिल सके।"
inputs = tokenizer.encode(hed,art, return_tensors='pt')
summary_ids = model.generate(inputs, max_length=500, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:")
print(summary)

Generated Summary:
सभी समृद्धि के लिए महत्वपूर्ण कदम है आजकल की तेजी से बदलती जीवनशैली में पर्यावरण संरक्षण के महत्वपूर्ण कदम आजकल की तेजी से बदलती जीवनशैली में पर्यावरण संरक्षण के महत्वपूर्ण कदम है। अव्यवस्थित और अति उपयोग से प्राकृतिक संसाधनों की दिक्कतें बढ़ रही हैं। अव्यवस्थित और अति उपयोग से प्राकृतिक संसाधनों की दिक्कतें बढ़ रही हैं।


In [ ]:
a='भारतकल के तेजी से बदलते वैज्ञानिक प्रसारों में, मानव ग्रीष्मकालीन ऊष्मा का अध्ययन एक महत्वपूर्ण और रोचक क्षेत्र बन चुका है। वैज्ञानिक प्रसारों में, मानव ग्रीष्मकालीन ऊष्मा का अध्ययन एक महत्वपूर्ण'
len(a)

In [26]:
!pip install evaluate
!pip install rouge_score
import evaluate
nltk.download('punkt')


rouge_score = evaluate.load("rouge")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:
from tqdm import tqdm
import numpy as np
from nltk.tokenize import sent_tokenize


generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets_valid,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )


all_preds = []
all_labels = []
for batch, labels in tqdm(tf_eval_dataset ):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 2158/2158 [09:10<00:00,  3.92it/s]


In [31]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value * 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 20.8333, 'rouge2': 0.0, 'rougeL': 20.8333, 'rougeLsum': 20.8333}

In [ ]:
# inputs = tokenizer("")
# inputs

hindi_text = "यह एक उदाहरण हिंदी पाठ है।"
tokens = tokenizer.encode(hindi_text, add_special_tokens=True)
tokens

In [ ]:
tokenizer.decode(tokens)

#---------------------------------------------------